In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold,cross_validate
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import itertools
from sklearn.impute import KNNImputer

In [ ]:

df = pd.read_csv('/home/pawblo/Desktop/studia/semestr3/Siwib/data/data/ae_retro_data.xlsx - Discretized.csv')
df.replace("?", np.nan, inplace=True)
df.head()

In [ ]:
nan_counts = df.isna().sum()


nan_counts = nan_counts[nan_counts > 0]
nan_counts_sorted = nan_counts.sort_values(ascending=False)

plt.figure(figsize=(10, 8))
nan_counts_sorted.plot(kind='bar')
plt.title('Number of NaN Values in Each Column (Sorted)')
plt.xlabel('Columns')
plt.ylabel('Number of NaN Values')
plt.xticks(rotation=90)
plt.show()

In [ ]:

df['CORR_CATEGORY'] = df['CORR_CATEGORY'].map({'LONG': 1, 'ADMIT': 1, 'SHORT': 0})

df['CORR_CATEGORY'].value_counts(), df.head()

In [ ]:

X = df.drop('CORR_CATEGORY', axis=1)
y = df['CORR_CATEGORY']


missing_values_count = X.isnull().sum()


missing_values_count_sorted = missing_values_count.sort_values(ascending=False)

columns_to_drop = missing_values_count_sorted.head(1).index


# X = X.drop(columns=columns_to_drop)

# # X = pd.get_dummies(X, drop_first=True)

# imp = SimpleImputer(strategy="most_frequent")
# X_filled = imp.fit_transform(X)
# # imputer = KNNImputer(n_neighbors=5)
# # X_filled = imputer.fit_transform(X)

# X = pd.DataFrame(X_filled, columns=X.columns)
# categorical_cols = X.columns
# categorical_cols
# print(X.shape)
# categorical_transformer = ColumnTransformer(transformers=[
#     ('cat', OneHotEncoder(), categorical_cols)
# ], remainder='passthrough') 
# X = categorical_transformer.fit_transform(X)


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X.apply(lambda x: x.astype(str)))

imputer = KNNImputer(n_neighbors=2)
X_imputed = imputer.fit_transform(X_encoded)

X = pd.DataFrame(X_imputed, columns=X.columns)


pca = PCA(n_components=12, whiten=True)
X_pca = pca.fit_transform(X_imputed)

X = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])])


In [ ]:

def make_prediction(y_pred_proba: np.ndarray, medium_risk: float, high_risk: float):
    return np.array([0 if response < medium_risk else 1 if response >= high_risk else np.nan for response in y_pred_proba])
def calculate_rates(
    y: np.ndarray, y_pred: np.ndarray, positives: float, negatives: float
) -> tuple[float, float, float, float]:
    tn, tp, fn, fp = confusion_matrix(y, y_pred).ravel()
    tpr = tp / (tp + fn)  
    fnr = fn / (fn + tp)  
    fpr = fp / (fp + tn)  
    tnr = tn / (tn + fp)  


    return tpr, fnr, fpr, tnr
def train_evaluate_classifier(classifier, X,y, name):
    auroc_scores = []
    auprc_scores = []
    medium = []
    high = []
    positives = []
    negatives = []
    fnrs = []
    fprs = []
    for _ in range(3):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random.randint(0,130))
        
        for train_index, test_index in skf.split(X, y):
           
            X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[test_index]
            y_train_fold, y_val_fold = y[train_index], y[test_index]
            classifier.fit(X_train_fold, y_train_fold)
            y_pred = classifier.predict(X_val_fold)
            auroc = roc_auc_score(y_val_fold, y_pred)
            auprc = average_precision_score(y_val_fold, y_pred)
            auroc_scores.append(auroc)
            auprc_scores.append(auprc)
            y_pred_proba = classifier.predict_proba(X_val_fold)[:,1]
            y_pred_proba_train = classifier.predict_proba(X_train_fold)[:, 1]
            fpr, tpr, thresholds = roc_curve(y_train_fold, y_pred_proba_train)
            medium_risk = thresholds[tpr >= 0.99][0]
            high_risk = thresholds[fpr >= (1 - 0.9)][0]
            predicted = make_prediction(y_pred_proba, medium_risk, high_risk)
            medium.append(medium_risk)
            high.append(high_risk)
            positive = np.sum(predicted == 1)
            negative = np.sum(predicted == 0)
            unknown = np.sum(np.isnan(predicted))
            number_of_samples: int = len(X_val_fold)
            positives.append(positive / number_of_samples)
            negatives.append(negative / number_of_samples)
            mask = ~np.isnan(predicted)
            tpr, fnr, fpr, tnr = calculate_rates(y_val_fold[mask], predicted[mask], positive, negative)
            fnrs.append(fnr)
            fprs.append(fpr)
    return {
            f"{name}- aurocs":np.mean(auroc_scores),
            f"{name}- auprcs":np.mean(auprc_scores),
            f"{name}- positives":np.mean(positives),
            f"{name}- negatives":np.mean(negatives),
            f"{name}- fnrs":np.mean(fnrs),
            f"{name}- fprs":np.mean(fprs)
        }
    

In [ ]:
X

positives

In [ ]:
y.shape

In [ ]:
classifier = DecisionTreeClassifier(random_state=42, max_depth=10)
train_evaluate_classifier(classifier,X,y, "base")


In [ ]:
classifier = RandomForestClassifier(random_state=42)
print(train_evaluate_classifier(classifier,X,y, "target"))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import itertools
import numpy as np

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

def param_combinations(params):
    keys, values = zip(*params.items())
    for v in itertools.product(*values):
        yield dict(zip(keys, v))

results = []

for params in param_combinations(param_grid):
    classifier = RandomForestClassifier(random_state=42, **params)
    evaluation_results = train_evaluate_classifier(classifier, X, y, "base")
    results.append({'params': params, 'evaluation_results': evaluation_results})

for result in results:
    print(result)

In [ ]:
x_points=[]
y_points=[]
labels = []
i=0
for res in results:
    x_points.append(res['evaluation_results']['base- aurocs']+res['evaluation_results']['base- auprcs'])
    y_points.append(res['evaluation_results']['base- fnrs']+res['evaluation_results']['base- fprs'])
    labels.append(i)
    i+=1

In [ ]:
plt.scatter(x_points, y_points)
for x in range(len(labels)):
    plt.text(x_points[x], y_points[x], labels[x])
plt.show()

In [ ]:
result_target=results[20]
classifier = DecisionTreeClassifier(random_state=42, max_depth=10)
result_base = train_evaluate_classifier(classifier,X,y, "base")
result_base

In [ ]:
result_target

In [ ]:
import pandas as pd

y_example = pd.Series(y)

class_counts = y_example.value_counts()

plt.bar(class_counts.index, class_counts.values, color=['blue', 'orange'])
plt.xlabel('Decision Class')
plt.ylabel('Count')
plt.title('Count of Decision Classes')
plt.xticks(ticks=class_counts.index, labels=['Class 0', 'Class 1'])
plt.show()